In [1]:
# %load "/home/foysal/onedrive/Northeastern University/Spring 2021/FIU ML/Data Generator/RAND-Lab-DataGenerator2.py"
#!/usr/bin/python3
# (C) Written by Udara De Silva (udesi001@fiu.edu)
import os
import numpy as np
import keras

from scipy.io import loadmat
from IPython.core.debugger import set_trace

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras.'

    def __init__(
        self,
        batch_size,
        data_path,
        num_samples_per_block=1024,
    ):
        'Initialization'
        self.batch_size = batch_size
        self.data_path =  '/home/foysal/ML/Noise_data'
        # A frame is a matlab file
        # Each frame has Y number of blocks with length X
        # X * Y < number of samples per frame
        self.num_frames_per_pattern=1000
        self.num_samples_per_frame=128000 # number of samples in one file
        self.num_blocks_per_frame = 1 # Y
        self.num_samples_per_block = num_samples_per_block # X
        self.num_rx_beams = 32 # channel count
        self.walk_patterns = ['P11', 'P12', 'P21', 'P22', 'P31', 'P41']
        self.num_patterns=len(self.walk_patterns)
        if not self.num_samples_per_block * self.num_blocks_per_frame <= self.num_samples_per_frame:
            print('[ERROR  ]: number of samples per block * number of blocks per frame > number of samples in a frame')


    def __len__(self):
        'Denotes the number of batches per epoch.'
        return int(self.num_patterns*(self.num_frames_per_pattern//np.ceil(self.batch_size/self.num_blocks_per_frame)))

    def __getitem__(self, index):
        'Generate one batch of data.'

        f_pattern = index % self.num_patterns
        # if batch size > num_blocks_per_frame we need to use several frames(files)
        f_trial   = int( (index//self.num_patterns) * np.ceil(self.batch_size // self.num_blocks_per_frame) + 1)

        file = os.path.join( self.data_path,  self.walk_patterns[f_pattern] + '_trial' + str(f_trial))

        block_index = 0
        out = np.zeros((self.batch_size, self.num_rx_beams, self.num_samples_per_block, 2)) # out(1000, 32, block_size, 2)
        for i in range(self.batch_size):
            data = self.next_block(file, block_index)
            if data is None:
                block_index = 0
                f_trial += 1
                file = os.path.join(self.data_path, self.walk_patterns[f_pattern] + '_trial' + str(f_trial))
                data = self.next_block(file, block_index)

            out[i, :, :, :] = np.stack((data, f_pattern * np.ones((self.num_rx_beams, self.num_samples_per_block))), axis=2)
            block_index += 1

        x = out[:,:,:,0]
        y = out[:,:,:,1]

        return x, y

    def next_block(self, matfile, block_index):
        if block_index + 1 > self.num_blocks_per_frame:
            return None

        mat = loadmat(matfile)
        data = mat['export_noise_mat']
        data = data[:,(block_index*self.num_samples_per_block):(block_index+1)*self.num_samples_per_block]
        return data


if __name__ == '__main__':
    dg = DataGenerator(1,'./Noise_data')

    print('Size of datagen : {}'.format(len(dg)))

    for (i, data) in enumerate(dg):
        print('Batch Index {}\n\n'.format(i))
        print(data[0][0, 0:5, 0:5])
        exit()


Size of datagen : 6000
Batch Index 0


[[ 20.  47.  43.  32.  25.]
 [ 19.  25.  45.  77.  85.]
 [ 10.   6.  21.  73.  91.]
 [ 20.  28.  49.  87.  97.]
 [ 39.  35.  67. 136. 154.]]
Batch Index 1


[[ 1. 25. 29. 12.  8.]
 [ 6. 21. 30. 29. 21.]
 [ 2. 11. 15. 15.  8.]
 [-1.  9. 11. 13. 15.]
 [12. 39. 51. 43. 26.]]
Batch Index 2


[[48. 41. 23. 24. 10.]
 [55. 61. 69. 70. 37.]
 [11. 19. 24. 32. 16.]
 [50. 44. 24. 19.  9.]
 [39. 46. 54. 60. 33.]]
Batch Index 3


[[ 4.  4. 18. 57. 50.]
 [ 9.  6. 11. 29. 27.]
 [ 5.  1.  3. 19. 16.]
 [ 1.  2.  5. 14.  7.]
 [ 6.  3.  8. 28. 25.]]
Batch Index 4


[[ 4. -2.  1.  5. -1.]
 [34. 44. 33. 21.  7.]
 [21. 42. 34. 16.  2.]
 [29. 30. 20. 10.  1.]
 [52. 84. 72. 38. 11.]]
Batch Index 5


[[37. 34. 11. -2.  8.]
 [21. 24. 19. 10. 24.]
 [11. 13.  6. -3.  3.]
 [12. 12.  8.  6. 17.]
 [47. 49. 30. 11. 22.]]
Batch Index 6


[[ 17.  45.  44.  30.  28.]
 [ 16.  23.  46.  75.  88.]
 [  7.   4.  22.  71.  94.]
 [ 17.  26.  50.  85. 100.]
 [ 36.  33.  68. 134. 157.]]
Bat